In [1]:
!pip install -U langchain
!pip install -U langchain_community
!pip install -U langchain_openai
!pip install faiss-cpu
!pip install pypdf

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.0 MB 6.8 MB/s eta 0:00:01
   ------------ --------------------------- 0.3/1.0 MB 5.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.0 MB 4.0 MB/s eta 0:00:01
   ------------------- -------------------- 0.5/1.0 MB 2.6 MB/s eta 0:00:01
   ------------------------- -------------- 0.6/1.0 MB 2.7 MB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 2.7 MB/s eta 0:00:01
   ------------------------------------- -- 0.9/1.0 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/411.6 kB ? eta -:--:--
   -------------- ------------------------- 153.6/411.6 kB 3.1 MB/s eta 0:00:01
   -----------------

In [1]:
## Fetching the openai key 
import os
import openai
openai.api_key = open("C:/Users/test/Desktop/ArMan/02 _ Manasa Documents/Educarion/AiI-ML 2023/NLP/Gen AI/OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [2]:
## Fetching the document 
from langchain_community.document_loaders import PyPDFLoader
pdfloader = PyPDFLoader("C:/Users/test/Desktop/ArMan/02 _ Manasa Documents/Educarion/AiI-ML 2023/NLP/NLP Syntactic Processing/SyntacticProcessing.pdf")
pdfpages = pdfloader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

In [3]:
## Document preperation
## Loading the documents using build in extractors, formatters, loaders, embeddings and LLMs in langchain
mydocuments = pdfloader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(mydocuments)

#Using openAI default embedding model
embeddings = OpenAIEmbeddings()

# Interacting with FAISS using API functions in langchain
db = FAISS.from_documents(texts,embeddings)

In [4]:
db

In [5]:
#Using langchain retriever

from langchain_core.tools import create_retriever_tool
retriever = db.as_retriever()
tool = create_retriever_tool(
    retriever,
    name = "Biomedical_Retriever",
    description="Searches and retrieves documents related to Biomedical based on famous book by author Khandpur"
)
tools = [tool]

#Wrapping LLM with conversational agents

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

# API by langchain to interact with chatmodels
from langchain_openai.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [6]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Define the custom prompt
custom_prompt = PromptTemplate(
    template="""You are an expert assistant who answers questions strictly based on the information provided in a document. 

Guidelines:
1. If the question can be answered using the document, provide a concise, accurate response.
2. If the question cannot be answered using the document or is unrelated, respond with: 
   "I'm sorry, the response to the query is not available at the moment.Please refine the question or contact the concerned department."

Document: {context}

Question: {question}

Answer:""",
    input_variables=["context", "question"],
)

llm = ChatOpenAI(temperature=0)
# Configure the retrieval-based QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,  
    retriever=retriever,  
    chain_type_kwargs={"prompt": custom_prompt},
)


In [7]:
del input

NameError: name 'input' is not defined

In [7]:
print("Welcome to chatbot MedXionic.AI! Ask me anything related to our current Product Development. Type 'exit' to quit.")

# Initialize chat history
chat_history = []

while True:
    # Get user input
    user_query = input("You: ")
    
    # Check for exit condition
    if user_query.lower() in ["exit", "quit"]:
        print("Have a nice day! Hope to assist you again soon!")
        break

    # Format the chat history for context
    history_context = "\n".join(
        [f"User: {entry['user']}\nBot: {entry['bot']}" for entry in chat_history]
    )
    
    # Combine history with the new question
    full_context = f"{history_context}\nUser: {user_query}\nBot:"
    
    # Get the response from the QA chain
    response = qa_chain.run(full_context)

    # Add the exchange to the history
    chat_history.append({"user": user_query, "bot": response})

    # Print the response
    print("Response:", response)


Welcome to chatbot MedXionic.AI! Ask me anything related to our current Product Development. Type 'exit' to quit.


You:  what is the meaning of Parsing?


C:\Users\test\AppData\Local\Temp\ipykernel_17264\1296213475.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(full_context)


Response: Syntactic analysis or parsing may be defined as the process of analysing the strings of symbols in natural language conforming to the rules of formal grammar.


You:  How is syntactic processing different from semantic processing?


Response: I'm sorry, the response to the query is not available at the moment. Please refine the question or contact the concerned department.


You:  exit


Have a nice day! Hope to assist you again soon!
